In [43]:
import json
from pyspark.sql import functions as F
from pyspark.sql import udf as udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, LongType, DoubleType, FloatType
from pyspark.sql import SparkSession
import datetime

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
import sys
!{sys.executable} -m pip install boto3
#!pip install boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

invalid syntax (<stdin>, line 2)
  File "<stdin>", line 2
    !{sys.executable} -m pip install boto3
    ^
SyntaxError: invalid syntax



In [37]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1582844154837_0001,pyspark,idle,Link,Link,✔


In [42]:
sc.install_pypi_package("boto3==1.12.9")
import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
from botocore.exceptions import ClientError

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.8.1  
boto                       2.49.0 
jmespath                   0.9.4  
lxml                       4.4.2  
mysqlclient                1.4.6  
nltk                       3.4.5  
nose                       1.3.4  
numpy                      1.14.5 
pip                        20.0.2 
py-dateutil                2.2    
python36-sagemaker-pyspark 1.2.6  
pytz                       2019.3 
PyYAML                     3.11   
setuptools                 45.2.0 
six                        1.13.0 
soupsieve                  1.9.5  
wheel                      0.34.2 
windmill                   1.6

### Process Song Data

In [57]:
sc._jsc.hadoopConfiguration().set("mapreduce.fileoutputcommitter.algorithm.version", "2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# get filepath to song data file
song_data ="s3n://udacity-dend/song_data/A/W/I/TRAWIAZ128F934AE04.json"
song_data ="s3n://udacity-dend/song_data/A/W/I/"

# read song data file
songs_data=spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession \
    .builder \
    .appName("project-4") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# extract columns to create songs table
songs_data.printSchema() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [12]:
# Process Song Data

songs_data.createOrReplaceTempView("songs_data_table")

# extract columns to create songs table (song_id, title, artist_id, year, duration)
songs_table = spark.sql('''SELECT song_id, title, artist_id, year, duration
                           FROM songs_data_table''')

# write songs table to parquet files
#songs_table.write.parquet('songs_table.parquet')

# extract columns to create artists table (artist_id, name, location, lattitude, longitude)
artists_table = spark.sql('''SELECT artist_id, artist_name AS name, artist_location AS location,
                                    artist_latitude AS latitude, artist_longitude AS longitude
                             FROM songs_data_table''')
   
# write artists table to parquet files
artists_table.write.parquet('artists_table.parquet')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
songs_table.write.parquet("s3n://de-nano-project-4/output/songs_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Process Log Data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
schema = StructType([
    StructField('artist',StringType(),True),
    StructField('auth',StringType(),True),
    StructField('firstName',StringType(),True),
    StructField('gender',StringType(),True),
    StructField('itemInSession',LongType(),True),
    StructField('lastName',StringType(),True),
    StructField('length',DoubleType(),True),
    StructField('level',StringType(),True),
    StructField('location',StringType(),True),
    StructField('method',StringType(),True),
    StructField('page',StringType(),True),
    StructField('registration',DoubleType(),True),
    StructField('sessionId',LongType(),True),
    StructField('song',StringType(),True),
    StructField('status',LongType(),True),
    StructField('ts',LongType(),True),
    StructField('userAgent',StringType(),True),
    StructField('userId',StringType(),True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Process Log Data

# get filepath to log data file
log_data = 's3a://udacity-dend/log_data/*/*/*.json'

# read log data file
log_data = spark.read.json(log_data, schema) 

log_data.createOrReplaceTempView("log_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
log_data.show()

In [15]:
log_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [16]:
# filter by actions for song plays
log_data = spark.sql('''SELECT *
             FROM log_table
             WHERE page = "NextSong"''')

log_data.createOrReplaceTempView("log_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# extract columns for users table (user_id, first_name, last_name, gender, level)
users_table = spark.sql('''SELECT DISTINCT userId, firstName, lastName, gender, level
                           FROM log_table''')
users_table.write.parquet('users_table.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# create datetime column from original timestamp column
timestamp_to_sec = F.udf(lambda x: x/1000, FloatType())
log_data = log_data.withColumn('start_time', timestamp_to_sec('ts'))
log_data = log_data.withColumn('datetime', F.from_unixtime('start_time'))
log_data.createOrReplaceTempView("log_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# extract columns to create time table (start_time, hour, day, week, month, year, weekday)
time_table = spark.sql('''SELECT DISTINCT start_time,
                    datetime, 
                    date_format(datetime, 'HH') AS hour,
                    date_format(datetime, 'dd') AS day,
                    date_format(datetime, 'w') AS week,
                    date_format(datetime, 'MM') AS month,
                    date_format(datetime, 'yyyy') AS year,
                    date_format(datetime, 'u') AS weekday 
             FROM log_table
          ''')

# write time table to parquet files partitioned by year and month
time_table.write.parquet('time_table.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# read in song data to use for songplays table
song_df = spark.read.parquet("songs_table.parquet")
song_df.createOrReplaceTempView("songs_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# read in artist data to use for songplays table
artist_df = spark.read.parquet("artists_table.parquet")
artist_df.createOrReplaceTempView("artists_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
song_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)

In [23]:
artist_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

In [31]:
# extract columns from joined song and log datasets to create songplays table 
# (songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
songplays_table = spark.sql('''SELECT CONCAT(e.ts, '-', e.sessionId) AS songplay_id, e.ts AS start_time, e.userId AS user_id,
                                    e.level AS level, sa.song_id AS song_id, 
                                    sa.artist_id AS artist_id, e.sessionId AS session_id, 
                                    e.location AS location, e.userAgent AS user_agent
                               FROM log_table AS e
                               JOIN (
                                   SELECT s.song_id, a.artist_id, s.title, a.name, s.duration
                                   FROM songs_table AS s
                                   INNER JOIN artists_table AS a
                                   ON s.artist_id = a.artist_id) AS sa
                               ON sa.title = e.song 
                               AND sa.name = e.artist 
                               AND sa.duration = e.length
                            ''')

songplays_table.write.parquet('songplays_table.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…